In [3]:
#import all tools for project
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import numpy as np
import datetime
import matplotlib.pyplot as plt
import re
from time import sleep

# Determining the Frequency of Gendered Language

### References
Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

Dr. Mattei, Tulane University. SimpleText Notebook. 2020. 
Bo Pang and Lillian Lee, A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts, Proceedings of ACL 2004.

In [11]:
#Import CSV containing mass amount of scraped reviews for analysis
hg = pd.read_csv("./revs_gendered.csv")
hg.dropna(inplace = True)
hg.columns = ['Review', 'Gender']
hg

,Review,Gender
0,"Exelent he really care about you, he is very p...",Male
1,Dr. A-Rahim was very knowledgeable about my lo...,Male
2,Horrible physician treats you like a kid. Pts ...,Female
3,Dr Aaberg has been a very good dr for my husba...,Male
4,Dr. Aaberg has been treating my macular degene...,Male
...,...,...
8889,Very helpful and sincere about my pain needs! ...,Male
8890,"Several of my children see Dr. Zach, she is a ...",Female
8891,Amazing Pediatric Neurologist stays up to date...,Female
8892,Dr. Zach has been a Godsend for our daughter! ...,Female


In [13]:
letters = pd.read_csv("./letters_tidied.csv")
letters.dropna(inplace = True)
letters.columns = ['Review', 'Gender']
letters['Gender'] = letters['Gender'].replace(0.0, 'Female')
letters['Gender'] = letters['Gender'].replace(1, 'Male')
letters

,Review,Gender
0,"dear review committee members,\r\n\r\nit is wi...",Female
1,this letter addresses some of my thoughts and ...,Female
2,I am writing this letter to recommend REDACTED...,Male
4,it is my sincere pleasure to nominate dr. REDA...,Female
5,i am writing to strongly recommend REDACTED RE...,Male
...,...,...
92,I am writing to highly recommend that you acce...,Female
93,i am writing to highly recommend that you acce...,Female
94,i am writing to highly recommend that you acce...,Female
95,\r\ni am writing to highly recommend that you ...,Female


In [20]:
# concatenate two dataframes (because i can't figure out whihc weight i'm supposed to change lolz)

allData = pd.concat([hg, letters])
allData

,Review,Gender
0,"Exelent he really care about you, he is very p...",Male
1,Dr. A-Rahim was very knowledgeable about my lo...,Male
2,Horrible physician treats you like a kid. Pts ...,Female
3,Dr Aaberg has been a very good dr for my husba...,Male
4,Dr. Aaberg has been treating my macular degene...,Male
...,...,...
92,I am writing to highly recommend that you acce...,Female
93,i am writing to highly recommend that you acce...,Female
94,i am writing to highly recommend that you acce...,Female
95,\r\ni am writing to highly recommend that you ...,Female


In [15]:
import sklearn
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [21]:
# Vectorize hg
vec = TfidfVectorizer(min_df = 2, 
                      max_df = 0.98, 
                      ngram_range=(1,1)) # play with min_df and max_df

reviews = allData['Review']

# transform this into a sparse vector!
vec.fit(reviews)
tf_idf_sparse = vec.transform(reviews)
tf_idf_sparse

#only 222 unique words due to min_df 0.03 --> change to 2, maybe change to 5 (could get meaningless words)

<8983x8350 sparse matrix of type '<class 'numpy.float64'>'
	with 341059 stored elements in Compressed Sparse Row format>

In [22]:
revs = dict(zip(allData.Review, allData.Gender))

In [23]:
# create list of cleaned, stemmed adjectives, for later analysis. 

vect = CountVectorizer()
vect.fit(revs)
words = vect.get_feature_names()

adjectives = []

stop_words = list(set(stopwords.words('english')))

# remove stopwords 
stopped = [w for w in words if not w in stop_words]
    
# parts of speech tagging for each word 
pos = nltk.pos_tag(stopped)

# make a list of  all adjectives identified by the allowed word types list above
for w in pos:
    if w[1][0] == 'J':
        adjectives.append(w[0].lower())

In [24]:
print(tf_idf_sparse[0, :])

  (0, 8308)	0.18087671068924527
  (0, 7963)	0.14515562205290422
  (0, 7385)	0.35561253933942405
  (0, 6038)	0.27852469718081846
  (0, 5793)	0.2373437823422639
  (0, 3983)	0.12189971460612926
  (0, 3374)	0.2636511185535702
  (0, 2703)	0.6446316283629488
  (0, 2326)	0.10587678031682882
  (0, 1571)	0.28720820096031574
  (0, 1248)	0.20441648315214442
  (0, 519)	0.0869707345247959
  (0, 197)	0.2070634329049377


In [25]:
# We can now use this to classify the reviews!! but we need to test/train split again.

# Split..
X_train, X_test, y_train, y_test = train_test_split(tf_idf_sparse, 
                                                    allData['Gender'], 
                                                    test_size=0.2)

In [26]:
logisticRegr = LogisticRegression(max_iter=100000, class_weight='balanced') 
model = logisticRegr.fit(X_train, y_train)

/Users/laurensussman/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [27]:
logisticRegr.classes_

array(['Female', 'Male'], dtype=object)

In [28]:
logisticRegr.coef_[:,0]

array([0.23844634])

In [29]:
# Make a dataframe with the words, coefficients, and classes...
recs = []

for w,i in vec.vocabulary_.items():
    recs.append([str(w)] + list(logisticRegr.coef_[:,i]))
# If we only have one class then we only get weight..
# df_weights = pd.DataFrame(tripples, columns=['word']+list(logisticRegr.classes_))
df_weights = pd.DataFrame(recs, columns=['word', 'weight'])

In [30]:
df_weights.sort_values('weight', ascending=False)[:25]

,word,weight
1,he,13.403489
86,his,8.171164
85,him,7.210929
329,surgery,3.081185
88,staff,1.884345
922,man,1.733318
1362,mother,1.703119
13,was,1.433253
87,outstanding,1.277648
411,pain,1.252576


In [31]:
#print only adjectives
#male adjectives
maleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=False)[:25]
maleAdjectives

,word,weight
87,outstanding,1.277648
67,good,0.992073
1114,arrogant,0.981111
4838,faber,0.921097
20,great,0.909375
680,respectful,0.907518
1247,total,0.907418
1058,pleased,0.889486
2303,kindness,0.797820
111,much,0.676001


In [32]:
df_weights.sort_values('weight', ascending=True)[:25]

,word,weight
43,she,-13.988681
397,her,-10.972279
4782,ma,-2.877061
7812,redacted,-2.123760
6156,kabbash,-1.950370
4510,eady,-1.878422
304,unprofessional,-1.461373
589,woman,-1.454312
6970,pabolu,-1.382532
2388,breast,-1.321711


In [33]:
#female adjectives
femaleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=True)[:25]
#maybe take out adjective specification
#vocabulary being pruned down too aggressively
femaleAdjectives

,word,weight
4510,eady,-1.878422
304,unprofessional,-1.461373
14,knowledgeable,-1.142530
3549,cabalona,-1.094032
701,smart,-1.043758
693,primary,-1.014789
832,new,-0.939429
482,wonderful,-0.934203
2649,pediatrician,-0.906382
5596,gal,-0.856504
